In [1]:
import os
import glob
import pandas as pd

In [2]:
# Define working directory
os.chdir(r"C:\Users\Cristina Bardan\Desktop\Repositories\FinalProject-Spotify\Original_data")

In [3]:
# Use glob to match the pattern ‘csv’
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

# Combine all files in the list and export as CSV
df = pd.concat([pd.read_csv(f) for f in all_filenames ])
df.reset_index(drop=True, inplace=True)

In [4]:
df2 = df.drop(['track', 'artist', 'uri'], axis=1)

In [5]:
#Validation of correlation between variables
import numpy as np 
df3=df2.corr()

In [ ]:
#1. Sin key - crih
#2. Sin Tempo - chris  / - purvi
#3. Sin speechiness / purvi  -grecia
#4. Sin chorus_hit  / grecia 
#5. Sin key, Tempo, speechiness, chorus_hit- heidy


In [6]:
# Sin tempo & speechiness
data = df2
X = data[['danceability', 'energy','key', 'loudness','mode', 'acousticness', 'instrumentalness', 'liveness','valence', 'duration_ms', 'time_signature','chorus_hit', 'sections']]  
y = data.target

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

## Data Preprocessing

In [8]:
#Scale features (X) using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler(feature_range=(0,1)).fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test) 

X_train_scaled

array([[0.78947368, 0.52283395, 0.81818182, ..., 0.8       , 0.19905999,
        0.06508876],
       [0.54048583, 0.83094117, 0.81818182, ..., 0.8       , 0.21633373,
        0.04142012],
       [0.5       , 0.66588373, 0.81818182, ..., 0.8       , 0.20480589,
        0.03550296],
       ...,
       [0.57692308, 0.36777999, 0.81818182, ..., 0.8       , 0.15844718,
        0.09467456],
       [0.41902834, 0.16070793, 0.63636364, ..., 0.8       , 0.200772  ,
        0.04733728],
       [0.29554656, 0.24873856, 0.81818182, ..., 0.6       , 0.12021911,
        0.06508876]])

One-hot encode the labels

In [9]:
#One-hot encode output labels (y)
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

y_train_categorical

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

## Creating and defining our Deep Learning Model Architecture

In [10]:
#Create a sequential model
from tensorflow.keras.models import Sequential
model = Sequential() 

from tensorflow.keras.layers import Dense
number_inputs = 13  

#Create hidden layers
model.add(Dense(units=14,activation='relu', input_dim=number_inputs))
model.add(Dense(units=120,activation='relu'))
model.add(Dense(units=80,activation='relu'))

#Create output layer
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax')) 

Number of Hidden Nodes 
https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw#:~:text=The%20number%20of%20hidden%20neurons,size%20of%20the%20input%20layer.

In [11]:
#Model Summary
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 14)                196       
_________________________________________________________________
dense_1 (Dense)              (None, 120)               1800      
_________________________________________________________________
dense_2 (Dense)              (None, 80)                9680      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 162       
Total params: 11,838
Trainable params: 11,838
Non-trainable params: 0
_________________________________________________________________


In [12]:
#Compile the Model
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

## Training the Model

In [13]:
#Training the Model
import tensorflow as tf

model.fit(X_train_scaled,y_train_categorical,epochs=500, batch_size=2000, shuffle=True, verbose=2)

Epoch 1/500
16/16 - 0s - loss: 0.6740 - accuracy: 0.5975
Epoch 2/500
16/16 - 0s - loss: 0.6168 - accuracy: 0.6710
Epoch 3/500
16/16 - 0s - loss: 0.5598 - accuracy: 0.7080
Epoch 4/500
16/16 - 0s - loss: 0.5378 - accuracy: 0.7259
Epoch 5/500
16/16 - 0s - loss: 0.5298 - accuracy: 0.7309
Epoch 6/500
16/16 - 0s - loss: 0.5238 - accuracy: 0.7362
Epoch 7/500
16/16 - 0s - loss: 0.5183 - accuracy: 0.7388
Epoch 8/500
16/16 - 0s - loss: 0.5170 - accuracy: 0.7411
Epoch 9/500
16/16 - 0s - loss: 0.5133 - accuracy: 0.7426
Epoch 10/500
16/16 - 0s - loss: 0.5115 - accuracy: 0.7433
Epoch 11/500
16/16 - 0s - loss: 0.5095 - accuracy: 0.7444
Epoch 12/500
16/16 - 0s - loss: 0.5067 - accuracy: 0.7472
Epoch 13/500
16/16 - 0s - loss: 0.5064 - accuracy: 0.7463
Epoch 14/500
16/16 - 0s - loss: 0.5045 - accuracy: 0.7491
Epoch 15/500
16/16 - 0s - loss: 0.5048 - accuracy: 0.7493
Epoch 16/500
16/16 - 0s - loss: 0.5042 - accuracy: 0.7495
Epoch 17/500
16/16 - 0s - loss: 0.5023 - accuracy: 0.7504
Epoch 18/500
16/16 - 0s

## Validation of the Model

In [14]:
#Evaluate the Model using the testing data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
    
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

322/322 - 0s - loss: 0.4964 - accuracy: 0.7618
Loss: 0.49638205766677856, Accuracy: 0.7617982029914856


## Saving the Trained Model

In [ ]:
# # Save the model
# Define working directory
#os.chdir(r"C:\Users\hguzm\Documents\000. Personal\Bootcamp\Proyectos\FinalProject-Spotify")
#model.save("spotify_DeepLearning_Model.h5")

## Loading a Model

In [ ]:
# Load the model  
from tensorflow.keras.models import load_model
# Define working directory
os.chdir(r"C:\Users\hguzm\Documents\000. Personal\Bootcamp\Proyectos\FinalProject-Spotify")
model = load_model("spotify_DeepLearning_Model.h5")

In [ ]:
model.summary()

## Validation of the Model

In [ ]:
#Evaluate the Model using the testing data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
    
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# #Making Predictions with new data
# new_data = np.array([[0.2, 0.3, 0.4,0.2, 0.3, 0.4,0.2, 0.3, 0.4,0.2, 0.3, 0.4,0.2, 0.3, 0.4]])  # AQUI IRIA INFORMACIÓN DEL API
# print(f"Predicted class: {model.predict_classes(new_data)}")